In [4]:
##########NO HYDROGENS, FULL PERIODIC* (or deteriums) PREP##########


########### code by Jessi Hoernschemeyer made together with PhD candidate Jesse Jones and Dr. Maria Andrea Mroginski, 
########### supported by Dr. Cecilia Clementi TU Biomodeling, and ChatGPT
########## © Summer 2023  - Summer 2025, Berlin ####################

###*Besides those which are confirmed not in any PDBS in the RCSB, and hydrogens
#   and momentarily, besides ionic nitrogen/oxygen

from utils import load_train_dir, inputs_directory
from pkparse import pkad2_parser as pkad_parser

import glob, os
from zipfile import ZipFile, ZIP_DEFLATED
import gzip
import numpy as np

"""TODO: explain the maps 

array([[b'1A', b'1A'],
       [b'2A2', b'2A2'],
       [b'3A', b'3A'],
       [b'4A', b'4A'],
       [b'5A1', b'5A1'],
       [b'6A5', b'6A5'],
       
       How only titratable have a val after chain."""

import os, time


def run(selection="all"):
    """This takes the intersection of the fixed and original gzipped pdb files which are on disk and wraps them in a try-except clause."""
    t0 = time.time()

    #data directories
    OUT_DIR = "../../PK-EGNN/benchmark_data/PKAD/inputs"    
    #OUT_ZIP = "../../data/pkegnn_dataset.zip"
    #MAP_DIR = "../../pkparse_OUTS/res_maps" #TODO dont make maps after its been done before and e.g. wanna develop cys bridges or add hydrogens or whatever
    FAIL_GZ = "pkad_badparse.gz" #log
    rcsb_paths     =np.char.array(glob.glob("../../../src_data/PKAD2/pdbs/*.gz"))
    modeled_paths   = rcsb_paths#np.char.array(glob.glob("../../src_data/pdbs/fixed/*.gz"))

    print("saving inputs to:",OUT_DIR)

    #oldpdbs, newpdbs = load_train_dir(modeled_paths,rcsb_paths)
    print("accessing source directories working if files printed:",rcsb_paths[0],modeled_paths[0])

    #make output dirs/files
    if not os.path.exists(OUT_DIR):
        os.mkdir(OUT_DIR)
    with gzip.open(FAIL_GZ, "wb") as f:
        f.close()
    #if not os.path.exists(MAP_DIR):
    #    os.mkdir(MAP_DIR)
    oldpdbs, newpdbs=rcsb_paths,rcsb_paths


    if selection == "all":
        indices = len(oldpdbs)
    elif type(selection) == int:
        indices = selection
    else: #individual pdbs
        newpdbs=np.char.array([f"../../../src_data/PKAD2/{selection}.pdb.gz"])
        oldpdbs=newpdbs#np.char.array([f"../../data/pdbs/rcsb/{selection}.pdb.gz"])

    for i in range(0,indices):
        try:
            pkad_parser(newpdbs[i], oldpdbs[i],save_dir=OUT_DIR).run()
        except Exception as e:
            pdb=newpdbs[i].split(".pdb.gz")[0][-4:].encode()
            print("error in", pdb, ":", e)
            with gzip.open(FAIL_GZ, "ab") as f:
                f.write(pdb)
                f.write(b"\n")
                continue

    #print("zipping entire db",(time.time() - t0)/60, "mins")

    
    print("data prep time:", (time.time() - t0)/60, "mins", indices, "pdbs")
run("all")

saving inputs to: ../../PK-EGNN/benchmark_data/PKAD/inputs
accessing source directories working if files printed: ../../../src_data/PKAD2/pdbs/135L.pdb.gz ../../../src_data/PKAD2/pdbs/135L.pdb.gz
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
error in b'1A2P' : b'ZN'
../../PK-EGNN/benchmark_data/PKAD/inputs
error in b'1A6K' : b'FE'
../../PK-EGNN/benchmark_data/PKAD/inputs
error in b'1A6M' : b'FE'
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
error in b'1B2V' : b'CA'
../../PK-EGNN/benchmark_data/PKAD/inputs
error in b'1B2X' : b'ZN'
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark_data/PKAD/inputs
../../PK-EGNN/benchmark

In [2]:
import polars as pl
from collections import OrderedDict
import gzip
import time
import subprocess

import pandas as pd
to = time.time()

# Read the full CSV, preserving all columns
#df = pl.read_csv("/home/pbuser/Desktop/Jessi/data_prep/pkas.csv", separator=";")
f="../../../src_data/PKAD2/PKAD2_DOWNLOAD.xlsx"
df = pd.read_excel(f).sort_values("PDB ID")
dd={1:df["PDB ID"],
   2:df["Res. Name"],
   3:df["Res. ID"],
   4:df["Chain"],
   5:df["Expt. pKa"]}

resi_data=np.vstack([dd[1],dd[2].to_numpy(), dd[3].to_numpy(),dd[4].to_numpy(),dd[5].to_numpy()]).T
resi_data

#d=df.to_dict()

import numpy as np
#a=np.array(list(d.values()))
#sorted_indices = np.argsort(a)
# Squeeze to remove the extra dimension
#a_flat = a.squeeze()               # Now shape becomes (12628148,)
#indices_flat = sorted_indices.squeeze()

# Use the flattened indices to retrieve sorted data
#sorted_data = a_flat[indices_flat]



#parts = np.char.partition(sorted_data, ";")

#pdbs=parts[:,0]
#resi_data=parts[:,2]
#arr = pdbs#np.array([...])  # your sorted data
arr=dd[1].to_numpy()
unique_vals, counts = np.unique(arr, return_counts=True)

badtargets=[]

print("unique pdbs gotten, starting parsing targets", (to - time.time())/60)
code = {"H":"0",
        "A":"1",
        "L":"2",
        "T":"3",
        "G":"4",
        "C":"5"}
fullcode={
    "HIS":"0", "ASP":"1", "LYS":"2",
    "TYR":"3", "GLU":"4", "CYS":"5", "ARG":b"6"
}
refpk = {
    "ASP": np.float32(3.79),
    "CYS": np.float32(8.67),
    "GLU": np.float32(4.20),
    "HIS": np.float32(6.74),
    "LYS": np.float32(10.46),
    "TYR": np.float32(9.59),
}

#
split_indices = np.cumsum(counts)[:-1]
subs=np.split(resi_data,split_indices)


manual_parse=[]

for pdbdata,pdb in zip(subs,unique_vals):
    #
    #par = np.char.rpartition(pdbdata, ";")
    #info = [np.char.split(np.atleast_1d(line[0]),";")[0] for line in par if line[0]]
    
    seq = pdbdata[:,1]
    mask=np.array([s in fullcode.keys() for s in seq],dtype=bool)
    seq=seq[mask]
    seqq = np.char.array([fullcode[s] for s in seq])
    #np.array([s in fullcode.keys() for s in seq])
    num = pdbdata[:,2]#
    num = np.char.array([str(r) for r in num])[mask]
    ch = np.char.array([str(c) for c in pdbdata[:,3]])[mask]
    ids= num + ch + seqq
    pks=np.char.array(pdbdata[:,4][mask])
    #ids=ids[mask]
    
    try:
        pks=[np.round(np.float32(p) - refpk[s],6) for p,s in zip(pdbdata[:,4],seq)]#[np.round(np.float32(p[2]) - refpk[i[1][0][0]],6) for i,p in zip(info,par) if not np.char.startswith(i[1],"CT") and not np.char.startswith(i[1],"N")]
    except:
        manual_parse.append(pdb)

#        np.savez_compressed(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{pdb}.npz",pks=pks,ids=ids)
        #np.load(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{pdb}.npz")
    #except Exception as e:
    #    badtargets.append(pdb)
    #    print(pdb)
    #    print(e)

#with gzip.open("/home/jrhoernschemeyer/Desktop/data_prep/badtargets.gz", "wb") as f:
#    for pdb in badtargets:
#        f.write(pdb.encode())
#        f.write(b" ")



print("finished", (time.time()-to)/60)

unique pdbs gotten, starting parsing targets -0.0259840726852417
finished 0.02764960527420044


In [20]:
seqs=[]
for pdb in glob.glob("../../data/pkparse_OUTS/PKAD/seqs/*.npy"):
    seqs.append(np.load(pdb))
from Bio import pairwise2
import pandas as pd

#seqs = df['seq'].tolist()

def pct_id(s1, s2):
    # globalxx: simple match=1, mismatch/gap=0
    aln = pairwise2.align.globalxx(str(s1), str(s2), one_alignment_only=True)[0]
    matches = aln[2]                # score = # of matches
    length = max(len(s1), len(s2))  # or use aln length: len(aln[0])
    return matches / length

sim_matrix = pd.DataFrame(index=seqs, columns=seqs, dtype=float)
for i, s1 in enumerate(seqs):
    for j, s2 in enumerate(seqs):
        sim_matrix.iloc[i, j] = pct_id(str(s1), str(s2))

print(sim_matrix)


IndexError: iloc cannot enlarge its target object

In [24]:
sim=[[pct_id(a, b) for b in seqs] for a in seqs[20]]
sim

[[0.3375,
  0.575,
  0.4375,
  0.4375,
  0.625,
  0.25,
  0.55,
  0.75,
  0.875,
  0.4,
  0.575,
  0.4,
  0.675,
  0.2875,
  0.975,
  0.6625,
  0.225,
  0.575,
  0.575,
  0.7625,
  1.0,
  0.225,
  0.55,
  0.5625,
  0.975,
  0.8625,
  0.45,
  0.375,
  0.3,
  0.7625,
  0.45,
  0.4375,
  0.6375,
  0.4375,
  0.425,
  0.5625,
  0.8,
  0.4375,
  0.2,
  0.525,
  0.2,
  0.25,
  0.35,
  0.35,
  0.4625,
  0.4625,
  0.7875,
  0.4,
  0.5625,
  0.3125,
  0.7625,
  0.725,
  0.425,
  0.3875,
  0.5625,
  0.6125,
  0.575,
  0.6625,
  0.5875,
  0.45,
  0.3625,
  0.4125,
  0.425,
  0.8,
  0.3125,
  0.35,
  0.65,
  0.275,
  0.35,
  0.8,
  0.525,
  0.3125,
  0.375,
  0.4375,
  0.4,
  0.35,
  0.475,
  0.3625,
  0.3375,
  0.4375,
  0.7625,
  0.8,
  0.375,
  0.7875,
  0.775,
  0.4875,
  0.275,
  0.275,
  0.45,
  0.3625,
  0.4625,
  0.475,
  0.4875,
  0.5625,
  0.6125,
  0.625,
  0.425,
  0.3125,
  0.4,
  0.3125,
  0.675,
  0.4625,
  0.4625,
  0.625,
  0.6875,
  0.3625,
  0.3625,
  0.5625,
  0.325,
  0.425,
  

In [132]:
len(unique_vals)
len(manual_parse)
unique_vals

array(['135L', '1A2P', '1A6K', '1A6M', '1A91', '1A93', '1ANS', '1AO6',
       '1AZP', '1B2V', '1B2X', '1BCX', '1BEG', '1BEO', '1BF4', '1BHC',
       '1BI6', '1BNI', '1BNJ', '1BNR', '1BNZ', '1BPI', '1BUS', '1BVI',
       '1C8C', '1CA5', '1CDC', '1CVO', '1D0D', '1DE3', '1DG9', '1DIV',
       '1DSB', '1DUK', '1DWR', '1EGF', '1EGO', '1EH6', '1EPG', '1EPH',
       '1EPI', '1ERA', '1ERT', '1ERU', '1EX3', '1EY0', '1FEZ', '1FKS',
       '1FMH', '1FNA', '1FW7', '1GB1', '1GOA', '1GS9', '1GYM', '1H4G',
       '1HHO', '1HIC', '1HNG', '1HPX', '1HRC', '1HV0', '1HV1', '1I0E',
       '1I0V', '1IG5', '1IGC', '1IGD', '1IGV', '1JAS', '1JBB', '1KF3',
       '1KXI', '1L54', '1LNI', '1LSE', '1LYS', '1LZ1', '1LZ3', '1MBC',
       '1MEK', '1MUT', '1NFN', '1NLX', '1NZP', '1P5F', '1PGA', '1PGB',
       '1PNT', '1POH', '1PPF', '1PPN', '1PPO', '1PTD', '1QH7', '1QLP',
       '1RDD', '1RGA', '1RGG', '1RNZ', '1SAP', '1STG', '1STN', '1THE',
       '1TRE', '1TRS', '1TRW', '1U2U', '1UBQ', '1WLA', '1XNB', '1XSN',
      

In [9]:
for p in unique_vals:
    !wget https://files.rcsb.org/download/{p}.pdb.gz -P "../../../src_data/PKAD2/pdbs"

--2025-07-22 00:14:16--  https://files.rcsb.org/download/135L.pdb.gz
Resolving files.rcsb.org (files.rcsb.org)... 65.9.66.92, 65.9.66.115, 65.9.66.124, ...
Connecting to files.rcsb.org (files.rcsb.org)|65.9.66.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26677 (26K) [application/octet-stream]
Saving to: ‘../../../src_data/PKAD2/pdbs/135L.pdb.gz’

135L.pdb.gz         100%[===================>]  26.05K  --.-KB/s    in 0.04s   

2025-07-22 00:14:16 (645 KB/s) - ‘../../../src_data/PKAD2/pdbs/135L.pdb.gz’ saved [26677/26677]

--2025-07-22 00:14:17--  https://files.rcsb.org/download/1A2P.pdb.gz
Resolving files.rcsb.org (files.rcsb.org)... 65.9.66.92, 65.9.66.115, 65.9.66.124, ...
Connecting to files.rcsb.org (files.rcsb.org)|65.9.66.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130768 (128K) [application/octet-stream]
Saving to: ‘../../../src_data/PKAD2/pdbs/1A2P.pdb.gz’

1A2P.pdb.gz         100%[===================>] 127.70K   6

In [125]:
pks

chararray([b'3.1', b'6.3', b'2', b'<2.0', b'3.1', b'2.1', b'3', b'2.2',
           b'3.6', b'3.75', b'3.3', b'7.75', b'3.8', b'4.2'], dtype='|S4')

In [64]:
np.subs

[array([['135L', 'ASP', 18, 'A'],
        ['135L', 'ASP', 52, 'A'],
        ['135L', 'ASP', 87, 'A'],
        ['135L', 'ASP', 119, 'A'],
        ['135L', 'GLU', 7, 'A'],
        ['135L', 'GLU', 35, 'A']], dtype=object),
 array([['1A2P', 'ASP', 8, 'C'],
        ['1A2P', 'C-term', 110, 'B'],
        ['1A2P', 'HIS', 102, 'C'],
        ['1A2P', 'ASP', 101, 'C'],
        ['1A2P', 'ASP', 93, 'C'],
        ['1A2P', 'ASP', 75, 'C'],
        ['1A2P', 'GLU', 73, 'C'],
        ['1A2P', 'GLU', 60, 'C'],
        ['1A2P', 'ASP', 54, 'C'],
        ['1A2P', 'ASP', 44, 'C'],
        ['1A2P', 'GLU', 29, 'C'],
        ['1A2P', 'ASP', 22, 'C'],
        ['1A2P', 'HIS', 18, 'C'],
        ['1A2P', 'ASP', 12, 'C'],
        ['1A2P', 'ASP', 86, 'C']], dtype=object),
 array([['1A6K', 'HIS', 116, 'A'],
        ['1A6K', 'HIS', 119, 'A'],
        ['1A6K', 'HIS', 113, 'A'],
        ['1A6K', 'HIS', 36, 'A'],
        ['1A6K', 'HIS', 93, 'A'],
        ['1A6K', 'HIS', 82, 'A'],
        ['1A6K', 'HIS', 81, 'A'],
        

{1: 0       1A2P
 1       1A2P
 2       1A2P
 3       1A2P
 4       1A2P
         ... 
 1451    7M2Z
 1452    7M2Z
 1453    7M2Z
 1454    7M2Z
 1455    7M2Z
 Name: PDB ID, Length: 1456, dtype: object,
 2: 0       ASP
 1       ASP
 2       HIS
 3       ASP
 4       GLU
        ... 
 1451    GLU
 1452    GLU
 1453    GLU
 1454    GLU
 1455    HIS
 Name: Res. Name, Length: 1456, dtype: object,
 3: 0        8
 1       12
 2       18
 3       22
 4       29
         ..
 1451     3
 1452    21
 1453    23
 1454    24
 1455    32
 Name: Res. ID, Length: 1456, dtype: object,
 4: 0       C
 1       C
 2       C
 3       C
 4       C
        ..
 1451    A
 1452    A
 1453    A
 1454    A
 1455    A
 Name: Chain, Length: 1456, dtype: object,
 5: 0        3.1
 1        3.8
 2       7.75
 3        3.3
 4       3.75
         ... 
 1451    4.32
 1452    4.68
 1453    4.67
 1454    4.64
 1455    7.16
 Name: Expt. pKa, Length: 1456, dtype: object}